In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
SportsList_url = 'https://en.wikipedia.org/wiki/Summer_Olympic_Games#List_of_Olympic_sports'
response = requests.get(SportsList_url)
soup = BeautifulSoup(response.text, 'html.parser')
SportsList_table = soup.findAll('td', attrs={'valign':'top'})

In [3]:
SportsList_parse = []
count = 0

# one table is made up of 2 tables next to each other
for i in SportsList_table:
    for sport in i.table.findAll('tr')[1:]:
        SportID = count
        SportName = sport.findAll('td')[0].a.text
        SportYears = sport.findAll('td')[1].text
        SportLink = "https://en.wikipedia.org" + sport.findAll('td')[0].a['href']
        SportsList_parse.append({'id':SportID,
                                 'sport':SportName,
                                 'active_years':SportYears,
                                 'wiki_link':SportLink})
        count += 1

In [4]:
SportsList_parse

[{'active_years': '1900–1908, 1920, since 1972',
  'id': 0,
  'sport': 'Archery',
  'wiki_link': 'https://en.wikipedia.org/wiki/Archery_at_the_Summer_Olympics'},
 {'active_years': 'All',
  'id': 1,
  'sport': 'Athletics',
  'wiki_link': 'https://en.wikipedia.org/wiki/Athletics_at_the_Summer_Olympics'},
 {'active_years': 'Since 1992',
  'id': 2,
  'sport': 'Badminton',
  'wiki_link': 'https://en.wikipedia.org/wiki/Badminton_at_the_Summer_Olympics'},
 {'active_years': '1992–2008, since 2020',
  'id': 3,
  'sport': 'Baseball',
  'wiki_link': 'https://en.wikipedia.org/wiki/Baseball_at_the_Summer_Olympics'},
 {'active_years': 'Since 1936',
  'id': 4,
  'sport': 'Basketball',
  'wiki_link': 'https://en.wikipedia.org/wiki/Basketball_at_the_Summer_Olympics'},
 {'active_years': '1900',
  'id': 5,
  'sport': 'Basque pelota',
  'wiki_link': 'https://en.wikipedia.org/wiki/Basque_pelota_at_the_1900_Summer_Olympics'},
 {'active_years': '1904, 1908, since 1920',
  'id': 6,
  'sport': 'Boxing',
  'wik

In [5]:
Olympic_years = list(range(1896,2017,4))
OlympicMedals_data = []

for year in Olympic_years:
    data = []
    Medals_url = 'https://en.wikipedia.org/wiki/'+str(year)+"_Summer_Olympics_medal_table"
    response = requests.get(Medals_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # finding medal count table, discarding first header row
    try:
        MedalCount_table = soup.find('table').findAll('tr')[1:]
        host = soup.find('table').find('tr', attrs={'bgcolor':'#CCCCFF'}).a.text
    except:
        pass
    
    # looping through medal count table by row 
    for row in MedalCount_table:
        # table formats are inconsistent, will need further work
        try:
            cell = row.findAll('td')
            rank = cell[0].text
            nation = cell[1].a.text
            gold = cell[2].text
            silver = cell[3].text
            bronze = cell[4].text
            data.append({'rank':rank,
                         'nation':nation,
                         'gold':gold,
                         'silver':silver,
                         'bronze':bronze})
        except:
            continue
        
    OlympicMedals_data.append({'year':year,'host':host,'data':data})
    print(str(year)+" is done")

1896 is done
1900 is done
1904 is done
1908 is done
1912 is done
1916 is done
1920 is done
1924 is done
1928 is done
1932 is done
1936 is done
1940 is done
1944 is done
1948 is done
1952 is done
1956 is done
1960 is done
1964 is done
1968 is done
1972 is done
1976 is done
1980 is done
1984 is done
1988 is done
1992 is done
1996 is done
2000 is done
2004 is done
2008 is done
2012 is done
2016 is done


In [6]:
# Creating df to manipulate
Olympics_df = pd.DataFrame(columns=['Year','Host','Rank','Nation','Gold_count','Silver_count','Bronze_count'])
index = 0

# Looping through created dictionary/json
for i in OlympicMedals_data:
    for ii in i['data']:
        Olympics_df.set_value(index,'Year',i['year'])
        Olympics_df.set_value(index,'Host',i['host'])
        try: 
            Olympics_df.set_value(index,'Rank',ii['rank'])
            Olympics_df.set_value(index,'Nation',ii['nation'])
            Olympics_df.set_value(index,'Gold_count',ii['gold'])
            Olympics_df.set_value(index,'Silver_count',ii['silver'])
            Olympics_df.set_value(index,'Bronze_count',ii['bronze'])
        except:
            pass
        index += 1

In [7]:
Olympics_df.groupby('Year').count()

,Host,Rank,Nation,Gold_count,Silver_count,Bronze_count
Year,,,,,,
1896,11,11,11,11,11,11
1900,21,21,21,21,21,21
1904,8,8,8,8,8,8
1908,18,18,18,18,18,18
1912,17,17,17,17,17,17
1920,20,20,20,20,20,20
1924,22,22,22,22,22,22
1928,25,25,25,25,25,25
1932,24,24,24,24,24,24
